## Deliverable 3. Create a Travel Itinerary Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Using the existing Vacation DB file load a df and use Google maps to display  vacation cities choices

In [3]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,Pension Maro'i
1,11,Atuona,PF,80.91,overcast clouds,-9.8000,-139.0333,Villa Enata
2,18,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
3,19,Arraial Do Cabo,BR,80.60,clear sky,-22.9661,-42.0278,Pousada Porto Praia
4,20,Vaini,TO,84.20,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort


In [71]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [72]:
# 4a. Add a marker layer for each city to the map.
# center=(30.0, 31.0) - historical geographic center
# center=(40.75, -74.00) - NYC, the true center of the world
fig = gmaps.figure(center=(40.75, -74.00), zoom_level=3.0)
heat_layer = gmaps.heatmap_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Plot the travel itinerary by mapping the route

In [75]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Rio De Janeiro"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Rio De Janeiro"]

vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Seropedica"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Sao Joao Da Barra"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Arraial Do Cabo"]

In [43]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end   = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

In [55]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation_drive = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        stroke_color='red',
        travel_mode='DRIVING')
vacation_bike = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        stroke_color='green',
        travel_mode='BICYCLING')
vacation_walk = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        stroke_color='blue',
        travel_mode='BICYCLING')
fig.add_layer(vacation_drive)
# you can overlay these as well by uncommenting 
#fig.add_layer(vacation_bike)
#fig.add_layer(vacation_walk)
fig

Figure(layout=FigureLayout(height='420px'))

### Merge the stops into the itinerary_df and use Google maps to display the selected cities 

In [57]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,33,Rio De Janeiro,BR,84.2,clear sky,-22.9028,-43.2075,Hotel Málaga
1,545,Seropedica,BR,82.4,clear sky,-22.7439,-43.7075,Pousada Vila do Celso
2,522,Sao Joao Da Barra,BR,80.6,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
3,19,Arraial Do Cabo,BR,80.6,clear sky,-22.9661,-42.0278,Pousada Porto Praia


In [64]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [73]:
# 11a. Add a marker layer for each city to the map.
# center=(30.0, 31.0) - historical geographic center
# center=(40.75, -74.00) - NYC, the true center of the world
# center=(-22.9661, -42.0278 - Arraial Do Cabo, trip center
fig = gmaps.figure(center=(-22.9661, -42.0278), zoom_level=7.5)
heat_layer = gmaps.heatmap_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))